In [90]:
pip install bs4


Note: you may need to restart the kernel to use updated packages.


In [91]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot
from bs4 import BeautifulSoup as soup
from datetime import date, datetime

In [92]:
today = datetime.now()

yesterday_str = "%s %d %d" %(date.today().strftime("%b"),today.day-1,today.year)
yesterday_str

'Jul 3 2021'

In [93]:
##Web Scraping
from urllib.request import Request, urlopen
url = "https://www.worldometers.info/coronavirus/#countries"
req = Request(url,headers={'User-Agent':"Mozilla/5.0"})

webpage = urlopen(req)
page_soup = soup(webpage, "html.parser")

In [94]:
table = page_soup.findAll("table",{"id":"main_table_countries_yesterday"})
 
containers = table[0].findAll("tr",{"style":""})
title = containers[0]
del containers[0]

all_data = []
clean = True
for country in containers:
    country_data = []
    country_container = country.findAll("td")
    
    if country_container[1].text=="China":
        continue
    for i in range(1, len(country_container)):
        final_feature = country_container[i].text
        if clean:
            if i!= 1 and i!= len(country_container)-1:
                final_feature = final_feature.replace(",","")
                if final_feature.find('+') != -1:
                    final_feature = final_feature.replace("+","")
                    final_feature = float(final_feature)
                elif final_feature.find("-") != -1:
                    final_feature = final_feature.replace("-","")
                    final_feature = float(final_feature)*-1
        if final_feature == 'N/A':
            final_feature = 0
        elif final_feature == "" or final_feature==" ":
            final_feature = -1
        country_data.append(final_feature)
    all_data.append(country_data)


In [95]:
df = pd.DataFrame(all_data)
df.drop([15,16,17,18,19,20],inplace = True, axis =1)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,World,184222453,381130.0,3987180,7214.0,168600170,330329.0,11635103,77794,23634,511.5,-1,-1,-1,All
1,USA,34588176,7978.0,621255,96.0,29087421,14540.0,4879500,3798,103884,1866,507601876,1524558,332950223,North America
2,India,30544485,43296.0,402015,947.0,29650169,52431.0,492301,8944,21918,288,416416463,298805,1393605337,Asia
3,Brazil,18742025,54556.0,523699,1631.0,17033808,44457.0,1184518,8318,87549,2446,54003112,252262,214075074,South America
4,France,5783654,3006.0,111152,17.0,5629530,2831.0,42972,1102,88410,1699,93999106,1436888,65418552,Europe


In [96]:
column_labels = ["Country","Total Cases", "New Cases","Total Deaths", "New Deaths", "Total Recovered", "Newly Recovered","Active Cases",
                "Serious/Crtical","Total Cases/1M", "Deaths/1M","Total Tests", "Test/1M","Population","Continent"]
df.columns = column_labels

In [97]:
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Newly Recovered,Active Cases,Serious/Crtical,Total Cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent
0,World,184222453,381130.0,3987180,7214.0,168600170,330329.0,11635103,77794,23634,511.5,-1,-1,-1,All
1,USA,34588176,7978.0,621255,96.0,29087421,14540.0,4879500,3798,103884,1866,507601876,1524558,332950223,North America
2,India,30544485,43296.0,402015,947.0,29650169,52431.0,492301,8944,21918,288,416416463,298805,1393605337,Asia
3,Brazil,18742025,54556.0,523699,1631.0,17033808,44457.0,1184518,8318,87549,2446,54003112,252262,214075074,South America
4,France,5783654,3006.0,111152,17.0,5629530,2831.0,42972,1102,88410,1699,93999106,1436888,65418552,Europe


In [98]:
#string to numeric conversion of columns
for label in df.columns:
    if label != 'Country' and label != 'Continent':
        df[label] = pd.to_numeric(df[label])

In [99]:
df["%Inc Cases"] = df["New Cases"] / df["Total Cases"]*100
df["%Inc Deaths"] = df["New Deaths"] / df["Total Deaths"]*100
df["%Inc Recovered"] = df["Newly Recovered"] / df["Total Recovered"]*100
df.head()

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Newly Recovered,Active Cases,Serious/Crtical,Total Cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
0,World,184222453,381130.0,3987180,7214.0,168600170,330329.0,11635103,77794,23634.0,511.5,-1,-1,-1,All,0.206886,0.180930,0.195924
1,USA,34588176,7978.0,621255,96.0,29087421,14540.0,4879500,3798,103884.0,1866.0,507601876,1524558,332950223,North America,0.023066,0.015453,0.049987
2,India,30544485,43296.0,402015,947.0,29650169,52431.0,492301,8944,21918.0,288.0,416416463,298805,1393605337,Asia,0.141747,0.235563,0.176832
3,Brazil,18742025,54556.0,523699,1631.0,17033808,44457.0,1184518,8318,87549.0,2446.0,54003112,252262,214075074,South America,0.291089,0.311438,0.260993
4,France,5783654,3006.0,111152,17.0,5629530,2831.0,42972,1102,88410.0,1699.0,93999106,1436888,65418552,Europe,0.051974,0.015294,0.050288


In [100]:
df.columns

Index(['Country', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths',
       'Total Recovered', 'Newly Recovered', 'Active Cases', 'Serious/Crtical',
       'Total Cases/1M', 'Deaths/1M', 'Total Tests', 'Test/1M', 'Population',
       'Continent', '%Inc Cases', '%Inc Deaths', '%Inc Recovered'],
      dtype='object')

In [101]:
##EDA
cases = df[["Total Recovered", "Active Cases","Total Deaths"]].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"]= np.round(100*cases_df['Total']/np.sum(cases_df["Total"]),2)
cases_df["virus"] = ["Covid-19" for i in range(len(cases_df))]
cases_df

,Type,Total,Percentage,virus
0,Total Recovered,168600170,91.52,Covid-19
1,Active Cases,11635103,6.32,Covid-19
2,Total Deaths,3987180,2.16,Covid-19


In [102]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.offline as py
import seaborn as sns
import gc
import warnings
warnings.filterwarnings("ignore")
from pandas_profiling import ProfileReport

In [103]:
fig = px.bar(cases_df,x="virus",y="Percentage",color ="Type",hover_data=["Total"])
fig.show()

In [104]:
df.columns

Index(['Country', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths',
       'Total Recovered', 'Newly Recovered', 'Active Cases', 'Serious/Crtical',
       'Total Cases/1M', 'Deaths/1M', 'Total Tests', 'Test/1M', 'Population',
       'Continent', '%Inc Cases', '%Inc Deaths', '%Inc Recovered'],
      dtype='object')

In [105]:
##EDA 2
cases = df[["New Cases", "Newly Recovered","New Deaths"]].loc[0]
cases_df = pd.DataFrame(cases).reset_index()
cases_df.columns = ["Type","Total"]

cases_df["Percentage"]= np.round(100*cases_df['Total']/np.sum(cases_df["Total"]),2)
cases_df["virus"] = ["Covid-19" for i in range(len(cases_df))]
cases_df


,Type,Total,Percentage,virus
0,New Cases,381130.0,53.03,Covid-19
1,Newly Recovered,330329.0,45.96,Covid-19
2,New Deaths,7214.0,1.00,Covid-19


In [106]:
fig = px.bar(cases_df,x="virus",y="Percentage",color ="Type",hover_data=["Total"])
fig.show()

In [107]:
per = np.round(df[["%Inc Cases", "%Inc Deaths", "%Inc Recovered"]].loc[0],2)
per_df = pd.DataFrame(per)
per_df.columns = ["Percentage"]
fig=go.Figure()

fig.add_trace(go.Bar(x=per_df.index,y=per_df['Percentage'],marker_color=["Yellow","Blue","Red"]))
fig.show()

# CONTINENT

In [108]:
#Continent
continent_df = df.groupby("Continent").sum().drop("All")
continent_df = continent_df.reset_index()
continent_df

,Continent,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Newly Recovered,Active Cases,Serious/Crtical,Total Cases/1M,Deaths/1M,Total Tests,Test/1M,Population,%Inc Cases,%Inc Deaths,%Inc Recovered
0,Africa,5683860,42945.0,145325,446.0,4923944,28998.0,614591,4556,639817.0,9108.0,53704863,5094118,1372638020,6.799973,-132.031902,-0.352906
1,Asia,56192580,142374.0,794441,2408.0,53526226,130690.0,1871912,26589,1525297.0,14951.3,838763706,36996675,3204193466,26.179912,-49.419471,21.701102
2,Australia/Oceania,75353,431.0,1277,-3.0,68626,126.0,5449,26,77594.0,591.0,23800200,1735869,41388712,6.181846,102.169103,10.122189
3,Europe,48196914,62849.0,1105635,843.0,45551680,39091.0,1539599,6502,3566319.0,69421.0,978559621,86238743,748083140,3.853663,-115.064369,2.654384
4,North America,40705422,24346.0,920369,405.0,34192813,25160.0,5592238,10843,1281100.0,18186.0,569913173,28735889,593457321,8.436365,-115.360369,12.998668
5,South America,33273680,108105.0,1015426,3009.0,28526706,106166.0,1861505,29209,806495.0,22340.0,134470626,5056458,434319157,4.090310,3.860103,5.772282


In [109]:
#function
def continent_visualization(v_list):
    for label in v_list:
        c_df = continent_df[['Continent',label]]
        c_df['Percentage'] = np.round(100*c_df[label]/np.sum(c_df[label]),2)
        c_df["virus"] = ["Covid-19" for i in range(len(c_df))]
        
        fig = px.bar(c_df,x="virus",y="Percentage",color ="Continent",hover_data=[label])
        fig.update_layout(title = {"text":f"{label}"})
        fig.show()
        gc.collect()

In [110]:
cases_list = ["Total Cases","Active Cases","New Cases","Serious/Crtical","Total Cases/1M"]
deaths_list = ["Total Deaths","New Deaths","Deaths/1M"]
recovered_list = ["Total Recovered","Newly Recovered","%Inc Recovered"]

In [111]:
continent_visualization(cases_list)

# COUNTRY #

In [112]:
#Country
df = df.drop([len(df)-1])
country_df = df.drop([0])

country_df

,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Newly Recovered,Active Cases,Serious/Crtical,Total Cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
1,USA,34588176,7978.0,621255,96.0,29087421,14540.0,4879500,3798,103884.0,1866.0,507601876,1524558,332950223,North America,0.023066,0.015453,0.049987
2,India,30544485,43296.0,402015,947.0,29650169,52431.0,492301,8944,21918.0,288.0,416416463,298805,1393605337,Asia,0.141747,0.235563,0.176832
3,Brazil,18742025,54556.0,523699,1631.0,17033808,44457.0,1184518,8318,87549.0,2446.0,54003112,252262,214075074,South America,0.291089,0.311438,0.260993
4,France,5783654,3006.0,111152,17.0,5629530,2831.0,42972,1102,88410.0,1699.0,93999106,1436888,65418552,Europe,0.051974,0.015294,0.050288
5,Russia,5585799,24439.0,137262,697.0,5053417,17899.0,395120,2300,38260.0,940.0,150700000,1032211,145997323,Europe,0.437520,0.507788,0.354196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Dominica,194,-1.0,-1,-1.0,191,-1.0,3,-1,2688.0,-1.0,19197,266012,72166,North America,-0.515464,100.000000,-0.523560
202,New Caledonia,129,-1.0,-1,-1.0,58,-1.0,71,-1,448.0,-1.0,34704,120395,288250,Australia/Oceania,-0.775194,100.000000,-1.724138
203,Anguilla,111,2.0,-1,-1.0,109,-1.0,2,-1,7333.0,-1.0,24074,1590408,15137,North America,1.801802,100.000000,-0.917431
204,Macao,54,-1.0,-1,-1.0,51,-1.0,3,-1,82.0,-1.0,4568,6940,658200,Asia,-1.851852,100.000000,-1.960784


In [113]:
LOOK_AT = 10
country = country_df.columns[1:14]
fig = go.Figure()
c=0
for i in country_df.index:
    if c<LOOK_AT:
        fig.add_trace(go.Bar(name=country_df['Country'][i],x=country, y=country_df.loc[i][1:14]))
    else:
        break
    c+=1
    
    
fig.update_layout(title={"text":f'top {LOOK_AT} countries affected'},yaxis_type = "log")
fig.show()